In [1]:
# TODO make a notebook with it
from transformers import AutoTokenizer, AutoModel
from pathlib import Path
import tensorflow as tf
import polars as pl
import os

#
from ebrec.utils._constants import (
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_IMPRESSION_ID_COL,
    DEFAULT_SUBTITLE_COL,
    DEFAULT_LABELS_COL,
    DEFAULT_TITLE_COL,
    DEFAULT_USER_COL,
)

#
from ebrec.utils._behaviors import (
    create_binary_labels_column,
    sampling_strategy_wu2019,
    add_known_user_column,
    add_prediction_scores,
    truncate_history,
)
from ebrec.utils._articles import convert_text2encoding_with_transformers
from ebrec.utils._polars import concat_str_columns, slice_join_dataframes
from ebrec.utils._articles import create_article_id_to_value_mapping
from ebrec.utils._nlp import get_transformers_word_embeddings
from ebrec.utils._python import write_submission_file, rank_predictions_by_score


from ebrec.models.newsrec.model_config import hparams_npa
from ebrec.models.newsrec.dataloader import LSTURDataLoader
from ebrec.models.newsrec.npa import NPAModel
import numpy as np



# Clear session to reset the GPU state
tf.keras.backend.clear_session()


# Set memory growth to true for all GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


c:\Users\stefa\.conda\envs\rsebrec\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_base = Path("../../data/processed/")


df_train_articles = pl.scan_parquet(data_base / "train_articles.parquet")
df_test_articles = pl.scan_parquet(data_base / "test_articles.parquet")

df_train = pl.scan_parquet(data_base / "train.parquet").collect()
df_validation = pl.scan_parquet(data_base / "validation.parquet").collect()
df_test = pl.scan_parquet(data_base / "test.parquet").collect()

In [3]:
TRANSFORMER_MODEL_NAME = "FacebookAI/xlm-roberta-base"
TEXT_COLUMNS_TO_USE = [DEFAULT_SUBTITLE_COL, DEFAULT_TITLE_COL]
MAX_TITLE_LENGTH = 30

# LOAD HUGGINGFACE:
transformer_model = AutoModel.from_pretrained(TRANSFORMER_MODEL_NAME)
transformer_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

# We'll init the word embeddings using the
word2vec_embedding = get_transformers_word_embeddings(transformer_model)
#

df_articles = df_train_articles.collect()
#df_articles_test = df_test_articles.collect()

df_articles, cat_cal = concat_str_columns(df_articles, columns=TEXT_COLUMNS_TO_USE)
df_articles, token_col_title = convert_text2encoding_with_transformers(
    df_articles, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH
)
# =>
article_mapping = create_article_id_to_value_mapping(
    df=df_articles, value_col=token_col_title
)
#
# df_articles_test, cat_cal = concat_str_columns(df_articles_test, columns=TEXT_COLUMNS_TO_USE)
# df_articles_test, token_col_title = convert_text2encoding_with_transformers(
#     df_articles_test, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH
# )
# # =>
# article_mapping_test = create_article_id_to_value_mapping(
#     df=df_articles_test, value_col=token_col_title
#)

c:\Users\stefa\.conda\envs\rsebrec\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
BATCH_SIZE = 16
user_id_mapping = {user_id: i for i, user_id in enumerate(df_train[DEFAULT_USER_COL].unique())}

print("Initiating the dataloaders")
train_dataloader = LSTURDataLoader(
    user_id_mapping=user_id_mapping,
    behaviors=df_train,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=False,
    batch_size=BATCH_SIZE,
)
val_dataloader = LSTURDataLoader(
    user_id_mapping=user_id_mapping,
    behaviors=df_validation,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=True,
    batch_size=BATCH_SIZE,
)

# test_dataloader = LSTURDataLoader(
#     user_id_mapping=user_id_mapping,
#     behaviors=df_test,
#     article_dict=article_mapping_test,
#     unknown_representation="zeros",
#     history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
#     eval_mode=True,
#     batch_size=BATCH_SIZE,
# )

Initiating the dataloaders


In [5]:
from tensorflow.keras import mixed_precision
# Equivalent to the two lines above
mixed_precision.set_global_policy('mixed_float16')


The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [6]:
# class hparams_npa:
#     # INPUT DIMENTIONS:
#     title_size: int = DEFAULT_TITLE_SIZE
#     history_size: int = 50
#     n_users: int = 50000
#     # MODEL ARCHITECTURE
#     cnn_activation: str = "relu"
#     attention_hidden_dim: int = 200
#     user_emb_dim: int = 400
#     filter_num: int = 400
#     window_size: int = 3
#     # MODEL OPTIMIZER:
#     optimizer: str = "adam"
#     loss: str = "cross_entropy_loss"
#     dropout: float = 0.2
#     learning_rate: float = 0.0001




from tensorflow.keras.losses import CategoricalCrossentropy
def model_builder(hp):
    hparams_npa.cnn_activation = hp.Choice('cnn_activation', ['relu', 'tanh'])
    hparams_npa.optimizer = hp.Choice('optimizer', ['adam', 'sgd'])
    hparams_npa.dropout = hp.Float('dropout', 0.1, 0.3)
    hparams_npa.learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    
    npa_model = NPAModel(
    hparams=hparams_npa,
    word2vec_embedding=word2vec_embedding,
    seed=42,
    )
    model, scorer = npa_model._build_graph()

    model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=hparams_npa.learning_rate),
    loss=CategoricalCrossentropy(),
    metrics=['accuracy']
    )
    
    return model

In [7]:
import keras_tuner as kt
# Your existing model setup and training code
MODEL_NAME = "NPA-monday-14-00"
LOG_DIR = f"./runs/{MODEL_NAME}"
MODEL_WEIGHTS = f"runs/{MODEL_NAME}/{MODEL_NAME}.weights.h5"


HISTORY_SIZE = 10

hparams_npa.history_size = HISTORY_SIZE



# Compile the model



tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
tuner.search(train_dataloader, validation_data = val_dataloader, epochs=50, callbacks=[stop_early])
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(best_hps)
# print(f"""
# The hyperparameter search is complete. The optimal number of units in the first densely-connected
# layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
# is {best_hps.get('learning_rate')}.
# """)


# # Train the model
# hist = model.fit(
#     train_dataloader,
#     validation_data=val_dataloader,
#     epochs=10,
#     batch_size = 16,
#     callbacks=[tensorboard_callback, early_stopping, modelcheckpoint],
#     verbose = 1
# )

# Load the best model weights
#model.load_weights(filepath=MODEL_WEIGHTS)


Reloading Tuner from my_dir\intro_to_kt\tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
20                |5                 |history_size
tanh              |relu              |cnn_activation
adam              |sgd               |optimizer
mean_squared_error|mean_squared_error|loss
0.23646           |0.15158           |dropout
0.0001            |0.0001            |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


 10/147 [=>............................] - ETA: 11:06 - loss: 1.7254 - accuracy: 0.2937